In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs


import os

# Webscrape Lotto Data

#### Data Webscraping

In [ ]:
import requests
import pandas as pd

url = 'http://www.nlcbplaywhelotto.com/nlcb-lotto-plus-results/'

MONTH = ['Jan','Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep','Oct','Nov','Dec']

YEAR = ['{:02d}'.format(i) for i in range(0,23)]

DATA = []

for year in YEAR:
    for month in MONTH:
        params = {
            'monthyear' : f'{month}-{year}'
        }
        r = requests.get(url = url , params=params)
        
        try:
            df = pd.read_html(r.content)[0].dropna(how='all')
            dates = df.iloc[::2].reset_index().drop(columns=['index']).rename(columns={'Draw#':'Date'})['Date'] # Even Rows
            Data = df.iloc[1::2].reset_index().drop(columns=['index']) # Odd Rows
            table = pd.merge(dates,Data,left_index=True, right_index=True).copy()
            
            DATA.append(table)
            print('[*] Data Scraped : ',params['monthyear'])
        except Exception as e:
            print(f'[*] Error {e} Occurred : {month}-{year}')
            continue
        
df = pd.concat(DATA)
df

### Data Cleaning

In [39]:
table = df.copy()

def clean_jp(row):
    try:
        return float(str(row).replace('$','').replace(',',''))
    except : return 0

table['Jackpot']    = table['Jackpot'].apply(clean_jp)
table['Date']       = pd.to_datetime(table['Date'], dayfirst=True)
table['Draw#']      = table['Draw#'].astype(int)
table['Power Ball']      = table['Power Ball'].astype(int)
table


,Date,Draw#,Numbers,Power Ball,Multiplier,Jackpot,Wins
0,2001-07-04,1,4-10-11-12-18,2,NaN,2691380.55,0
1,2001-07-07,2,2-10-11-26-28,10,NaN,3140281.24,0
2,2001-07-11,3,5-10-11-13-15,10,NaN,3428785.45,1
3,2001-07-14,4,9-11-24-26-33,4,NaN,1000000.00,0
4,2001-07-18,5,8-11-12-19-31,10,NaN,1000000.00,0
...,...,...,...,...,...,...,...
4,2022-12-17,2181,11-18-20-23-34,6,20,7718465.56,1
5,2022-12-21,2182,2-10-21-22-24,1,3,2000000.00,0
6,2022-12-24,2183,19-21-25-29-30,2,5,2000000.00,0
7,2022-12-28,2184,7-13-17-18-32,2,20,2000000.00,0


In [40]:
print('[*] Unique Values in Multiplier : ', table.Multiplier.unique())
table.Multiplier = table.Multiplier.fillna(-1)
table['Multiplier']      = table['Multiplier'].astype(int)
print('[*] New Unique Values in Multiplier : ', table.Multiplier.unique())

print('[*] Unique Values in Column : ', table.Wins.unique())
table.Wins = table.Wins.str.replace("X",'-1')
table['Wins']      = table['Wins'].astype(int)
table

print('[*] New Unique Values in Wins : ', table.Wins.unique())


table

[*] Unique Values in Multiplier :  [nan '4' '10' '20' '5' '3']
[*] New Unique Values in Multiplier :  [-1  4 10 20  5  3]
[*] Unique Values in Column :  ['0' '1' '2' 'X' '8' '3']
[*] New Unique Values in Wins :  [ 0  1  2 -1  8  3]


,Date,Draw#,Numbers,Power Ball,Multiplier,Jackpot,Wins
0,2001-07-04,1,4-10-11-12-18,2,-1,2691380.55,0
1,2001-07-07,2,2-10-11-26-28,10,-1,3140281.24,0
2,2001-07-11,3,5-10-11-13-15,10,-1,3428785.45,1
3,2001-07-14,4,9-11-24-26-33,4,-1,1000000.00,0
4,2001-07-18,5,8-11-12-19-31,10,-1,1000000.00,0
...,...,...,...,...,...,...,...
4,2022-12-17,2181,11-18-20-23-34,6,20,7718465.56,1
5,2022-12-21,2182,2-10-21-22-24,1,3,2000000.00,0
6,2022-12-24,2183,19-21-25-29-30,2,5,2000000.00,0
7,2022-12-28,2184,7-13-17-18-32,2,20,2000000.00,0


#### Data Exporting

In [41]:
table.to_excel('Lotto Scraped Data.xlsx')

In [15]:
import requests
import pandas as pd

month = 'Dec'
year = 2003
url = 'http://www.nlcbplaywhelotto.com/nlcb-lotto-plus-results/'
params = {
    'monthyear' : f'{month}-{str(year)[:2]}'
}
r = requests.get(url = url , params=params)
if r.status_code == 200:
    df = pd.read_html(r.content)
else:
    print('[*] Error Occured : ', r.status_code)
df[0]

,Draw#,Numbers,Power Ball,Multiplier,Jackpot,Wins
0,02-Dec-20,02-Dec-20,02-Dec-20,02-Dec-20,02-Dec-20,02-Dec-20
1,1994,5-7-12-19-27,4,NaN,"$2,000,000.00",0
2,05-Dec-20,05-Dec-20,05-Dec-20,05-Dec-20,05-Dec-20,05-Dec-20
3,1995,6-15-20-21-22,6,NaN,"$2,000,000.00",0
4,09-Dec-20,09-Dec-20,09-Dec-20,09-Dec-20,09-Dec-20,09-Dec-20
5,1996,10-14-22-26-32,10,NaN,"$2,267,330.45",0
6,12-Dec-20,12-Dec-20,12-Dec-20,12-Dec-20,12-Dec-20,12-Dec-20
7,1997,8-21-26-32-33,9,NaN,"$2,809,353.46",0
8,16-Dec-20,16-Dec-20,16-Dec-20,16-Dec-20,16-Dec-20,16-Dec-20
9,1998,9-17-19-27-31,6,NaN,"$3,304,314.11",0


In [19]:
df[0].iloc[::2].reset_index().drop(columns=['index']).rename(columns={'Draw#':'Date'})['Date']

0    02-Dec-20
1    05-Dec-20
2    09-Dec-20
3    12-Dec-20
4    16-Dec-20
5    19-Dec-20
6    23-Dec-20
7    26-Dec-20
8    30-Dec-20
Name: Date, dtype: object

In [21]:
df[0].iloc[1::2].reset_index().drop(columns=['index']) # Odd Rows

,Draw#,Numbers,Power Ball,Multiplier,Jackpot,Wins
0,1994,5-7-12-19-27,4,NaN,"$2,000,000.00",0
1,1995,6-15-20-21-22,6,NaN,"$2,000,000.00",0
2,1996,10-14-22-26-32,10,NaN,"$2,267,330.45",0
3,1997,8-21-26-32-33,9,NaN,"$2,809,353.46",0
4,1998,9-17-19-27-31,6,NaN,"$3,304,314.11",0
5,1999,8-11-16-19-31,9,NaN,"$3,986,673.96",0
6,2000,8-11-16-23-25,1,NaN,"$4,667,144.51",0
7,2001,11-16-17-33-35,1,NaN,"$5,236,073.71",0
8,2002,8-14-15-24-29,6,NaN,"$6,098,524.81",0


In [26]:

dates = df[0].iloc[::2].reset_index().drop(columns=['index']).rename(columns={'Draw#':'Date'})['Date'] # Even Rows
Data = df[0].iloc[1::2].reset_index().drop(columns=['index']) # Odd Rows
table = pd.merge(dates,Data,left_index=True, right_index=True).copy()
def clean_jp(row):
    try:
        return float(str(row).replace('$','').replace(',',''))
    except : return 0
    
table['Jackpot'] = table['Jackpot'].apply(clean_jp)
table['Date'] = pd.to_datetime(table['Date'], dayfirst=True)

table

,Date,Draw#,Numbers,Power Ball,Multiplier,Jackpot,Wins
0,02-Dec-20,1994,5-7-12-19-27,4,NaN,"$2,000,000.00",0
1,05-Dec-20,1995,6-15-20-21-22,6,NaN,"$2,000,000.00",0
2,09-Dec-20,1996,10-14-22-26-32,10,NaN,"$2,267,330.45",0
3,12-Dec-20,1997,8-21-26-32-33,9,NaN,"$2,809,353.46",0
4,16-Dec-20,1998,9-17-19-27-31,6,NaN,"$3,304,314.11",0
5,19-Dec-20,1999,8-11-16-19-31,9,NaN,"$3,986,673.96",0
6,23-Dec-20,2000,8-11-16-23-25,1,NaN,"$4,667,144.51",0
7,26-Dec-20,2001,11-16-17-33-35,1,NaN,"$5,236,073.71",0
8,30-Dec-20,2002,8-14-15-24-29,6,NaN,"$6,098,524.81",0


In [28]:
def clean_jp(row):
    try:
        return float(str(row).replace('$','').replace(',',''))
    except : return 0
    
table['Jackpot'] = table['Jackpot'].apply(clean_jp)
table['Date'] = pd.to_datetime(table['Date'], dayfirst=True)

table

,Date,Draw#,Numbers,Power Ball,Multiplier,Jackpot,Wins
0,2020-12-02,1994,5-7-12-19-27,4,NaN,2000000.00,0
1,2020-12-05,1995,6-15-20-21-22,6,NaN,2000000.00,0
2,2020-12-09,1996,10-14-22-26-32,10,NaN,2267330.45,0
3,2020-12-12,1997,8-21-26-32-33,9,NaN,2809353.46,0
4,2020-12-16,1998,9-17-19-27-31,6,NaN,3304314.11,0
5,2020-12-19,1999,8-11-16-19-31,9,NaN,3986673.96,0
6,2020-12-23,2000,8-11-16-23-25,1,NaN,4667144.51,0
7,2020-12-26,2001,11-16-17-33-35,1,NaN,5236073.71,0
8,2020-12-30,2002,8-14-15-24-29,6,NaN,6098524.81,0


In [ ]:

Data = 
df = pd.merge(dates,Data,left_index=True, right_index=True).copy()

In [ ]:
Year = ['{:02d}'.format(i) for i in range(1,24)]
Year

In [12]:
bs(r.content,'html').table.findAll('tr')[2]


<tr class="lotto-tr">
<td>2082</td>
<td>3-7-12-17-27</td>
<td class="powerball-td">2</td>
<td>
									NA								</td>
<td>$7,817,158.08</td><td>0</td> </tr>

In [19]:

# Parse the HTML content using BeautifulSoup
soup = bs(r.content, 'html')

# Find the table containing the results
table = soup.table

# Find the row containing the most recent result
rows = table.findAll('tr')
recent_row = rows[0]

# Extract the draw date and winning numbers
draw_date = recent_row.find_all('td')[0].text.strip()
winning_numbers = recent_row.find_all('td')[1].text.strip()

IndexError: list index out of range

In [20]:
recent_row

<tr><th>Draw#</th><th>Numbers</th><th>Power Ball</th><th>Multiplier</th><th>Jackpot</th><th>Wins</th></tr>

In [21]:
rows

[<tr><th>Draw#</th><th>Numbers</th><th>Power Ball</th><th>Multiplier</th><th>Jackpot</th><th>Wins</th></tr>,
 <tr class="lotto-date-tr"><td colspan="6"><strong>01-Jan-22</strong></td></tr>,
 <tr class="lotto-tr">
 <td>2082</td>
 <td>3-7-12-17-27</td>
 <td class="powerball-td">2</td>
 <td>
 									NA								</td>
 <td>$7,817,158.08</td><td>0</td> </tr>,
 <tr class="lotto-date-tr"><td colspan="6"><strong>05-Jan-22</strong></td></tr>,
 <tr class="lotto-tr">
 <td>2083</td>
 <td>3-16-17-21-23</td>
 <td class="powerball-td">6</td>
 <td>
 									NA								</td>
 <td>$8,710,484.94</td><td>0</td> </tr>,
 <tr class="lotto-date-tr"><td colspan="6"><strong>08-Jan-22</strong></td></tr>,
 <tr class="lotto-tr">
 <td>2084</td>
 <td>3-12-15-19-25</td>
 <td class="powerball-td">8</td>
 <td>
 									NA								</td>
 <td>$9,814,507.22</td><td>0</td> </tr>,
 <tr class="lotto-date-tr"><td colspan="6"><strong>12-Jan-22</strong></td></tr>,
 <tr class="lotto-tr">
 <td>2085</td>
 <td>6-11-14-15-33</td>
 <